# 1. Obtención de los datos

## 1.1. Web Scrapping

Este primer paso es el más importante y sin embargo el que podemos obviar de este notebook, pues se trata de la automatización de la descarga de los datos. Por ello, hay que tener claro con que bacteria vamos a trabajar. Nosotros sabemos, además, el tamaño que tiene aproximademente, y le podemos añadir un filtro.

In [144]:
from selenium import webdriver
import time
chrome_driver = "C:\\Users\\migue\\Neoland\\Python\\Selenium\\chromedriver.exe"

In [145]:
navegador = webdriver.Chrome(executable_path=chrome_driver)
navegador.get("https://www.ncbi.nlm.nih.gov/nuccore")
time.sleep(15)
navegador.find_element_by_id("term").send_keys("streptococcus pyogenes complete genome")
navegador.find_element_by_id('search').click()
time.sleep(15)
navegador.find_element_by_id('facet_rangeseqlen').click()
navegador.find_element_by_id("facet_range_stseqlen").send_keys("1500000")
navegador.find_element_by_id("facet_range_endseqlen").send_keys("2300000")
time.sleep(15)
navegador.find_element_by_id('facet_range_applyseqlen').click()
time.sleep(15)
navegador.find_element_by_link_text('20 per page').click()
time.sleep(15)
navegador.find_element_by_id('ps200').click()

EL archivo que nos interesa es FASTA, pero no todos los genomas están en dicho formato. Por eso, a la hora de seleccionar todos los genomas para la descarga en FASTA da error. Podemos hacer las checkboxes de toda la página automáticamente, pero hay que asegurarse página por página si hay archivos FASTA.

In [ ]:
checkboxes = navegador.find_elements_by_xpath("//input[@name='EntrezSystem2.PEntrez.Nuccore.Sequence_ResultsPanel.Sequence_RVDocSum.uid']")
for checkbox in checkboxes:
    checkbox.click()

# 2. Pre-procesamiento

## 2.1. Abriendo los archivos FASTA

In [1]:
import pandas as pd
from Bio import SeqIO
from os import listdir
from os.path import isfile, join
import os
os.chdir("C:\\Users\\migue\\Neoland\\Proyectos\\SpyAR\\data\\genomes")

In [2]:
files = [f for f in listdir("C:\\Users\\migue\\Neoland\\Proyectos\\SpyAR\\data\\genomes") if isfile(join("C:\\Users\\migue\\Neoland\\Proyectos\\SpyAR\\data\\genomes",f))]

In [3]:
for seq_record in SeqIO.parse(files[0], "fasta"):
        print(seq_record.id)
        print(len(seq_record))
        print(repr(seq_record.seq))
        print(seq_record.description)
        break

En el mismo orden que hemos mostrado la información, la vamos a agregar a una lista "**genomas**".

In [4]:
genomas = []

In [5]:
for seq_record in SeqIO.parse("C:\\Users\\migue\\Neoland\\Proyectos\\SpyAR\\data\\genomes\\sequence.fasta", "fasta"):
    genomas.append([seq_record.id,len(seq_record),seq_record.seq,seq_record.description])

Ahora podemos hacer un dataframe de pandas con nuestra lista de genomas:

In [6]:
df = pd.DataFrame(genomas, columns=['ID','Pb','Secuencia','Descripción'])

Aquí vamos a limpiar nuestra descripción para quedarnos sólo con las cepas...

In [7]:
for genoma in genomas:
    genoma[3] = genoma[3].replace(genoma[0]+' ','')
    print(genoma[3])

Streptococcus pyogenes MGAS8232, complete genome
Streptococcus dysgalactiae subsp. equisimilis GGS_124 DNA, complete genome
Streptococcus dysgalactiae subsp. equisimilis RE378 DNA, complete genome
Streptococcus pyogenes MGAS315, complete genome
Streptococcus equi subsp. zooepidemicus MGCS10565, complete genome
Streptococcus pyogenes DNA, complete genome, strain: M3-b
Streptococcus pyogenes strain TSPY208 chromosome, complete genome
Streptococcus pyogenes strain emm22.8 chromosome, complete genome
Streptococcus pyogenes strain emm25 chromosome, complete genome
Streptococcus pyogenes strain emm78.3 chromosome, complete genome
Streptococcus pyogenes strain TSPY556 chromosome, complete genome
Streptococcus pyogenes strain emm230 chromosome, complete genome
Streptococcus pyogenes strain emm97.1 chromosome, complete genome
Streptococcus pyogenes strain emm89.14 chromosome, complete genome
Streptococcus pyogenes strain emm123 chromosome, complete genome
Streptococcus pyogenes strain emm105 ch

In [8]:
spy = []
for genoma in genomas:
    genoma[3] = genoma[3].split(' ')
    if genoma[3][0] == 'Streptococcus' and genoma[3][1] == 'pyogenes':
        spy.append(genoma)
for genoma in spy:
    for i in range(0,len(genoma[3])):
        genoma[3][i] = genoma[3][i].replace(',','')
        genoma[3][i] = genoma[3][i].replace(' ','')

In [9]:
genomas = []
for genoma in spy:
    
    # para strain, conocemos su ubicación siempre [2]
    if genoma[3][2] == 'strain':
        genomas.append([genoma[0],genoma[3][3],genoma[1],genoma[2]])
        
    # cuando la cepa está al final
    elif genoma[3][2] == 'DNA':
        genomas.append([genoma[0],genoma[3][-1],genoma[1],genoma[2]])
        
    # para strain, conocemos su ubicación siempre [2]
    else:
        genomas.append([genoma[0],genoma[3][2],genoma[1],genoma[2]])

In [10]:
df = pd.DataFrame(genomas, columns=['ID','Cepa','Pb','Secuencia'])
df

,ID,Cepa,Pb,Secuencia
0,AE009949.1,MGAS8232,1895017,"(T, T, G, T, T, G, A, T, A, T, T, C, T, G, T, ..."
1,AE014074.1,MGAS315,1900521,"(T, T, G, T, T, G, A, T, A, T, T, C, T, G, T, ..."
2,AP014596.1,M3-b,1893821,"(T, T, G, T, T, G, A, T, A, T, T, C, T, G, T, ..."
3,CP033335.1,TSPY208,1801696,"(A, T, G, A, C, T, G, A, A, A, A, T, G, A, A, ..."
4,CP035438.1,emm22.8,1950616,"(A, T, G, A, C, T, G, A, A, A, A, T, G, A, A, ..."
...,...,...,...,...
501,NZ_JJKY01000001.1,ABC020021452,1812989,"(G, T, A, A, A, A, A, A, A, G, A, G, T, T, G, ..."
502,NZ_JJKV01000001.1,ABC020013256,1819209,"(T, T, G, T, T, G, A, T, A, T, T, C, T, G, T, ..."
503,NZ_JJKX01000001.1,ABC020017774,1814430,"(A, A, A, A, A, A, G, A, G, T, T, G, A, C, A, ..."
504,KK213748.1,ABC020056020,1846773,"(T, A, A, G, C, A, G, A, T, G, A, T, T, G, A, ..."


#### Export to genomas.csv

In [11]:
df.to_csv('C:\\Users\\migue\\Neoland\\Proyectos\\SpyAR\\data\\genomes\\genomas.csv',index = False,header=True)

## 2.2. BLASTn

El **BLAST** (Basic Local Alignment Search Tool) está considerado el Google de los biólogos, con esta herramienta se buscan secuencias concretas en los organismos de una base de datos. El NCBI dispone de un BLASTn (n por nucleótidos) para toda su base de datos, pero en este proyecto no es útil dado que sólo lo encuentra en 201 casos (de los 500 totales que disponemos). Por ello, vamos a recurrir a **UGENE**, un software bioinformático para hacer el BLAST con nuestra propia base de datos.

Tras hacer el BLAST, obtenemos los resultados en un archivo que guardamos como blast.csv y tiene lo siguientes atributos:

In [12]:
df = pd.read_csv("C:\\Users\\migue\\Neoland\\Proyectos\\SpyAR\\data\\blast\\blast.csv")
df

,Query Strand,Subject Start,Subject Stop,Subject Strand,Identity,Mismatches,Gaps,Subject Defline
0,-,1280088,1282343,+,100.0000,0,0,JJIM01000002.1 Streptococcus pyogenes ABC02005...
1,-,1349047,1351302,+,100.0000,0,0,KK213748.1 Streptococcus pyogenes ABC020056020...
2,-,1337849,1340104,+,100.0000,0,0,NZ_JJKX01000001.1 Streptococcus pyogenes ABC02...
3,-,1354307,1356562,+,100.0000,0,0,NZ_JJKV01000001.1 Streptococcus pyogenes ABC02...
4,-,1336508,1338763,+,100.0000,0,0,NZ_JJKY01000001.1 Streptococcus pyogenes ABC02...
...,...,...,...,...,...,...,...,...
495,-,1235521,1237776,+,99.3351,15,0,NZ_CP007240.1 Streptococcus pyogenes strain 7F...
496,+,453811,456066,+,99.3351,15,0,NZ_CP007562.1 Streptococcus pyogenes strain NG...
497,-,1235521,1237776,+,99.3351,15,0,"CP007240.1 Streptococcus pyogenes strain 7F7, ..."
498,+,453811,456066,+,99.3351,15,0,CP007562.1 Streptococcus pyogenes strain NGAS3...


In [13]:
blast = df.values.tolist()

Quiero comprobar que todos son Streptococcus pyogenes y están escritos más o menos en el mismo formato ('Streptococcus'), porque sólo nos queremos quedar con lo anterior, que es el ID del genoma.

In [14]:
for fila in blast:
    print(fila[7])

JJIM01000002.1 Streptococcus pyogenes ABC020056020 adVqh-supercont-complete.C2, whole genome shotgun sequence
KK213748.1 Streptococcus pyogenes ABC020056020 genomic scaffold adVqh-supercont-complete, whole genome shotgun sequence
NZ_JJKX01000001.1 Streptococcus pyogenes ABC020017774 adVrd-supercont1.1, whole genome shotgun sequence
NZ_JJKV01000001.1 Streptococcus pyogenes ABC020013256 adVrP-supercont1.1, whole genome shotgun sequence
NZ_JJKY01000001.1 Streptococcus pyogenes ABC020021452 adVrA-supercont1.1, whole genome shotgun sequence
NZ_JJKB01000001.1 Streptococcus pyogenes ABC020056068 adVqm-supercont1.1, whole genome shotgun sequence
NZ_JJJQ01000001.1 Streptococcus pyogenes ABC020048503 adVpA-supercont1.1, whole genome shotgun sequence
NZ_JJKC01000001.1 Streptococcus pyogenes ABC020059502 adVpW-supercont1.1, whole genome shotgun sequence
NZ_JJJY01000001.1 Streptococcus pyogenes ABC020048387 adVpR-supercont1.1, whole genome shotgun sequence
NZ_JJLB01000001.1 Streptococcus pyogenes A

In [15]:
for fila in blast:
    x = fila[7][:(fila[7].find(' Streptococcus'))]
    fila.append(x)
    fila.pop(7)

Ahora hay que unir los dataframes. Podemos hacerlo con pandas pero el tamaño es diferente, se ha perdido información durante el BLASTn. Así que vamos a ir uno a uno por **blast** comprobando si está en **genomas**.

In [16]:
print('Antes del BLASTn: ',len(genomas))
print('Después del BLASTn: ',len(blast))

Antes del BLASTn:  506
Después del BLASTn:  500


In [17]:
genomas = pd.read_csv("C:\\Users\\migue\\Neoland\\Proyectos\\SpyAR\\data\\genomes\\genomas.csv")
genomas_lista = genomas.values.tolist()

In [18]:
for genoma in genomas_lista:
    for x in blast:
        if genoma[0] == x[7]:
            genoma.append(x[0])
            genoma.append(x[1])
            genoma.append(x[2])
            genoma.append(x[3])
            genoma.append(x[4])
            genoma.append(x[5])
            genoma.append(x[6])
        else:
            pass

In [19]:
genomas = pd.DataFrame(genomas_lista, columns=['ID','CEPA','PB','SECUENCIA','HEBRA_GEN','CEPA_START','CEPA_STOP','HEBRA_CEPA','SIMILITUD','MISMATCHES','GAPS'])
genomas

,ID,CEPA,PB,SECUENCIA,HEBRA_GEN,CEPA_START,CEPA_STOP,HEBRA_CEPA,SIMILITUD,MISMATCHES,GAPS
0,AE009949.1,MGAS8232,1895017,TTGTTGATATTCTGTTTTTTCTTTTTTAGTTTTCCACATAAAAAAT...,-,1393952.0,1396207.0,+,99.6897,7.0,0.0
1,AE014074.1,MGAS315,1900521,TTGTTGATATTCTGTTTTTTCTTTTTTAGTTTTCCACATAAAAAAT...,-,1404400.0,1406655.0,+,99.6897,7.0,0.0
2,AP014596.1,M3-b,1893821,TTGTTGATATTCTGTTTTTTCTTTTTTAGTTTTCCACATAAAAAAT...,+,490191.0,492445.0,+,99.6454,7.0,1.0
3,CP033335.1,TSPY208,1801696,ATGACTGAAAATGAACAAATTTTTTGGAACAGGGTCTTGGAATTAG...,-,1337002.0,1339257.0,+,99.5124,11.0,0.0
4,CP035438.1,emm22.8,1950616,ATGACTGAAAATGAACAAATTTTTTGGAACAGGGTCTTGGAATTAG...,-,1446258.0,1448513.0,+,99.7340,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
501,NZ_JJKY01000001.1,ABC020021452,1812989,GTAAAAAAAGAGTTGACAATGACAGTAAAGATATCGTATACTAATA...,-,1336508.0,1338763.0,+,100.0000,0.0,0.0
502,NZ_JJKV01000001.1,ABC020013256,1819209,TTGTTGATATTCTGTTTTTTCTTTTTTAGTTTTCCACATGAAAAAT...,-,1354307.0,1356562.0,+,100.0000,0.0,0.0
503,NZ_JJKX01000001.1,ABC020017774,1814430,AAAAAAGAGTTGACAATGACAGTAAAGATATCGTATACTAATAAAG...,-,1337849.0,1340104.0,+,100.0000,0.0,0.0
504,KK213748.1,ABC020056020,1846773,TAAGCAGATGATTGATTTAAAAAGGAATCTTTTCAAAATTGATGTC...,-,1349047.0,1351302.0,+,100.0000,0.0,0.0


#### Export to genomas_blast.csv

In [20]:
genomas.to_csv('C:\\Users\\migue\\Neoland\\Proyectos\\SpyAR\\data\\genomes\\genomas_blast.csv',index=False,header=True)

## 2.3. Añadir las clases

In [21]:
genomas = pd.read_csv("C:\\Users\\migue\\Neoland\\Proyectos\\SpyAR\\data\\genomes\\genomas_blast.csv")
genomas_lista = genomas.values.tolist()

Desde que haya más de 1 mismatch... mutación!!

In [22]:
for i in range(0,len(genomas_lista)):
    if genomas_lista[i][9] > 0:
        genomas_lista[i].append(1)
    else:
        genomas_lista[i].append(0)

In [23]:
genomas_lista[0]

['AE009949.1',
 'MGAS8232',
 1895017,
 '...',
 '-',
 1393952.0,
 1396207.0,
 '+',
 99.6897,
 7.0,
 0.0,
 1]

In [24]:
df = pd.DataFrame(genomas_lista,columns=['ID','CEPA','PB','SECUENCIA','HEBRA_GEN','CEPA_START','CEPA_STOP','HEBRA_CEPA','SIMILITUD','MISMATCHES','GAPS','MUTACION'])
df

,ID,CEPA,PB,SECUENCIA,HEBRA_GEN,CEPA_START,CEPA_STOP,HEBRA_CEPA,SIMILITUD,MISMATCHES,GAPS,MUTACION
0,AE009949.1,MGAS8232,1895017,TTGTTGATATTCTGTTTTTTCTTTTTTAGTTTTCCACATAAAAAAT...,-,1393952.0,1396207.0,+,99.6897,7.0,0.0,1
1,AE014074.1,MGAS315,1900521,TTGTTGATATTCTGTTTTTTCTTTTTTAGTTTTCCACATAAAAAAT...,-,1404400.0,1406655.0,+,99.6897,7.0,0.0,1
2,AP014596.1,M3-b,1893821,TTGTTGATATTCTGTTTTTTCTTTTTTAGTTTTCCACATAAAAAAT...,+,490191.0,492445.0,+,99.6454,7.0,1.0,1
3,CP033335.1,TSPY208,1801696,ATGACTGAAAATGAACAAATTTTTTGGAACAGGGTCTTGGAATTAG...,-,1337002.0,1339257.0,+,99.5124,11.0,0.0,1
4,CP035438.1,emm22.8,1950616,ATGACTGAAAATGAACAAATTTTTTGGAACAGGGTCTTGGAATTAG...,-,1446258.0,1448513.0,+,99.7340,6.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
501,NZ_JJKY01000001.1,ABC020021452,1812989,GTAAAAAAAGAGTTGACAATGACAGTAAAGATATCGTATACTAATA...,-,1336508.0,1338763.0,+,100.0000,0.0,0.0,0
502,NZ_JJKV01000001.1,ABC020013256,1819209,TTGTTGATATTCTGTTTTTTCTTTTTTAGTTTTCCACATGAAAAAT...,-,1354307.0,1356562.0,+,100.0000,0.0,0.0,0
503,NZ_JJKX01000001.1,ABC020017774,1814430,AAAAAAGAGTTGACAATGACAGTAAAGATATCGTATACTAATAAAG...,-,1337849.0,1340104.0,+,100.0000,0.0,0.0,0
504,KK213748.1,ABC020056020,1846773,TAAGCAGATGATTGATTTAAAAAGGAATCTTTTCAAAATTGATGTC...,-,1349047.0,1351302.0,+,100.0000,0.0,0.0,0


## 2.4. Balancear las clases

In [25]:
genomas = df
genomas['MUTACION'].value_counts()

1    416
0     90
Name: MUTACION, dtype: int64

In [26]:
max_0 = genomas['MUTACION'].value_counts()[0]

Debido a la gran diferencia de nucleótidos entre cada cepa, vamos a elegir las que se acerquen más a la media.

In [27]:
media = genomas['PB'].mean()

Podemos añadir una columna con la resta de los nucleótidos totales y la media y seleccionar los valores más pequeños.

In [28]:
RESTA = []
for i in range(0,len(genomas)):
    x = float(genomas['PB'].iloc[[i]] - media)
    if x < 0:
        RESTA.append(x*-1)
    else:
        RESTA.append(x)
genomas['RESTA'] = RESTA

Filtramos y ordenamos para eliminar las que no nos interesan.

In [29]:
mutados = genomas[(genomas['MUTACION']==1)].sort_values(by='RESTA',ascending=True).head(max_0)
no_mutados = genomas[(genomas['MUTACION']==0)]
df = pd.concat([mutados, no_mutados])
df = df.sort_index()
df = df[['ID','PB','SECUENCIA','MUTACION','CEPA_START','CEPA_STOP','HEBRA_GEN']]

In [30]:
df

,ID,PB,SECUENCIA,MUTACION,CEPA_START,CEPA_STOP,HEBRA_GEN
5,CP035442.1,1835714,ATGACTGAAAATGAACAAATTTTTTGGAACAGGGTCTTGGAATTAG...,1,1349357.0,1351612.0,-
7,CP032700.1,1828132,ATGACTGAAAATGAACAAATTTTTTGGAACAGGGTCTTGGAATTAG...,1,471381.0,473636.0,+
8,CP035451.1,1826832,ATGACTGAAAATGAACAAATTTTTTGGAACAGGGTCTTGGAATTAG...,1,1370186.0,1372441.0,-
9,CP035447.1,1812090,ATGACTGAAAATGAACAAATTTTTTGGAACAGGGTCTTGGAATTAG...,1,1331195.0,1333450.0,-
13,CP035428.1,1816007,ATGACTGAAAATGAACAAATTTTTTGGAACAGGGTCTTGGAATTAG...,1,1349091.0,1351346.0,-
...,...,...,...,...,...,...,...
500,NZ_JJKB01000001.1,1822128,TTGTTGATATTCTGTTTTTTCTTTTTTAGTTTTCCACATGAAAAAT...,0,1357282.0,1359537.0,-
501,NZ_JJKY01000001.1,1812989,GTAAAAAAAGAGTTGACAATGACAGTAAAGATATCGTATACTAATA...,0,1336508.0,1338763.0,-
502,NZ_JJKV01000001.1,1819209,TTGTTGATATTCTGTTTTTTCTTTTTTAGTTTTCCACATGAAAAAT...,0,1354307.0,1356562.0,-
503,NZ_JJKX01000001.1,1814430,AAAAAAGAGTTGACAATGACAGTAAAGATATCGTATACTAATAAAG...,0,1337849.0,1340104.0,-


## 2.5. Alineamiento de secuencias

In [31]:
import numpy as np
import random as rd

In [32]:
df_lista = df.values.tolist()

In [33]:
pbp = []
for i in range(0,len(df_lista)):    
    
    # Los indices no funcionan con float
    df_lista[i][4] = int(df_lista[i][4])
    df_lista[i][5] = int(df_lista[i][5])
    secuencia = df_lista[i][2][(df_lista[i][4]):(df_lista[i][5])]
    pbp.append([df_lista[i][0],df_lista[i][1],secuencia,df_lista[i][3]])

ValueError: cannot convert float NaN to integer

Hay valores perdidos... vamos a comprobar donde están... con esto le estamos diciendo que nos devuelva el ID de aquellos con los valores perdidos y si es mutado o no (por el balance de clases).

In [34]:
for i in range(0,len(df_lista)):  
    if np.isnan(df_lista[i][4])==True:
        print(df_lista[i][0])
        print(df_lista[i][3])
        print('-------------')

CP049799.1
0
-------------
CP049800.1
0
-------------
NZ_CP049799.1
0
-------------
NZ_LS483414.1
0
-------------
NZ_LS483338.1
0
-------------
NZ_UHGZ01000002.1
0
-------------
LS483414.1
0
-------------
NZ_GL397225.1
0
-------------


Son todos de la clase no mutada, tiene sentido teniendo en cuenta que hemos clasificados en mutados y no mutados por el número de mismatches, eso quiere decir que en estos no ha habido alineamiento con el BLAST. Es decir, que no se ha encontrado la secuencia de la PBP.

Ahora tenemos que eliminarlas del dataframe y eliminar la misma cantidad de las mutadas. Primero las no mutadas...

In [41]:
x = []
for i in df_lista:  
    if np.isnan(i[4])==False:
        x.append(i)

El balance **antes**:

In [42]:
mutados = 0
no_mutados = 0
for i in x:
    if i[3] == 1:
        mutados +=1
    else:
        no_mutados +=1
print('Mutados: ', mutados)
print('No mutados: ', no_mutados)

Mutados:  90
No mutados:  82


In [43]:
i = 0
while i < (mutados-no_mutados):
    n = rd.randint(0,len(x))
    if x[n][3]==1:
        x.pop(n)
        i += 1
    else:
        continue

El balance **después**:

In [44]:
mutados = 0
no_mutados = 0
for i in x:
    if i[3] == 1:
        mutados +=1
    else:
        no_mutados +=1
print('Mutados: ', mutados)
print('No mutados: ', no_mutados)

Mutados:  82
No mutados:  82


## 2.6. Aislar secuencias PBP

El tamaño de las secuencias es de casi 2 millones. Requiere mucho poder computacional, no es viable...

In [45]:
pbp = []
for i in range(0,len(x)):
    # Los indices no funcionan con float
    x[i][4] = int(x[i][4])
    x[i][5] = int(x[i][5])
    secuencia = x[i][2][(x[i][4]):(x[i][5])]
    pbp.append([x[i][0],x[i][1],x[i][3],x[i][6],secuencia])

In [46]:
for i in pbp:
    print(i[3])
    print(i[4][0:10])
    print('-------------')

-
TAATCTCCTA
-------------
+
TGAAAAAATG
-------------
-
TAATCTCCTA
-------------
-
TAATCTCCTA
-------------
-
TAATCTCCTA
-------------
-
TAATCTCCTA
-------------
-
TAATCTCCTA
-------------
+
TGAAAAAATG
-------------
+
TGAAAAAATG
-------------
+
TGAAAAAATG
-------------
-
TAATCTCCTA
-------------
+
TGAAAAAATG
-------------
+
TGAAAAAATG
-------------
+
TGAAAAAATG
-------------
+
TGAAAAAATG
-------------
-
TAATCTCCTA
-------------
-
TAATCTCCTA
-------------
+
TGAAAAAATG
-------------
-
TAATCTCCTA
-------------
-
TAATCTCCTA
-------------
-
TAATCTCCTA
-------------
-
TAATCTCCTA
-------------
+
TGAAAAAATG
-------------
-
TAATCTCCTA
-------------
+
TGAAAAAATG
-------------
-
TAATCTCCTA
-------------
-
TAATCTCCTA
-------------
+
TGAAAAAATG
-------------
+
TGAAAAAATG
-------------
+
TGAAAAAATG
-------------
-
TAATCTCCTA
-------------
+
TGAAAAAATG
-------------
+
TGAAAAAATG
-------------
+
TGAAAAAATG
-------------
+
TGAAAAAATG
-------------
-
TAATCTCCTA
-------------
-
TAATCTCCTA
-------------
+

Vemos secuencias que no coinciden debido a la orientación de las hebras del alineamiento, hay que igualarlas para que nuestra red neuronal no de errores ni malas predicciones eliminando las de sentido **+**.

In [47]:
pbp2 = []
for i in range(0,len(pbp)):
    if pbp[i][3] == '-':
        pbp2.append(pbp[i])
    else:
        pass

## 2.7. Alineamiento de secuencias

Para el alineamiento se utilizó **ClustalOmega** junto a **UGENE**. ClustalOmega es otro software informático ampliamente utilizado en bioinformática para realizar alineamientos múltiples de secuencias. Antes, hubo que exportar los datos que preparamos en un archivo FASTA para poder correrlo con el programa.

In [48]:
fasta = ''
for i in pbp2:
    fasta += '>' + i[0] + "\n" + i[4] + "\n"

In [49]:
print(fasta)

>CP035442.1
TAATCTCCTAAAGTAATGGTCATTTTTTTTATTTTTTTCAAGGACTTACCAACATCAACACTTTGTTTCATAACACGACCAGAATCTGTTCCTTTAAAGCTGATGTCTATTCCAGTCCATTTAGCAAAGGTTTTAACATTGGATTTTGTCCAGCCATACATGTCTGGTACCTCCACAAAACGGTCTGATAATATGAGAACTTGTTGGTTTTCTGTTAACGTTTGACCGGGCTGATGCGATACTTTTTTGATCTTGCTGCCAGTACCAAGGACAACTGGCTGGACAAGATTTCGACGCAACTCGCTTGATGTCTCACCAGGATTCTTTCCTACAAAGTTTGGTAATTTATAAGTTGTTTGACGATTAGCATCTGATACTACTGGCTTAGTTAGTGTATCTTGCATTAAGTATGCTTCTTCCAATACTGGGTTAACCACATCTTGCCAAAAAAGGGGACCAAAATGTTGTGGTTTAGTCATAGTAACATACATCAAAAAGTCTGGTTTATCAGCTGGCACCATTGCCACAACCGAATAGACATAGTTAGTCAATCCACCATCTTGATAACCACTTCCATCTTCTGAACCAATTTGTGCTGTTCCTGATTTAACAGCAACAGGTAAATCACCCACTTTAATAATTGGTCCAAATGTTTTTGAATAGAGTGTTCCAAACTCAGGGTCTGTTCCTACACCAATCATGTACTGTCTTGTTTCACTAGCAGCTTTTTTTGATACAGGTTTTCCAACAATTTCTTTATTTGCCGTTCTAAAACTTGCTGTGTTAGGATCATAAATTTGACTGATAAATTGTGGCTCTAACATCTCACCATTATTAGAAATAGCAGTAAAGGCTCTAAGCATTTGAATCTGGGTTACAGAAATCCCTTGACCAAAAGCGCTCATAGCTTGAGTCACGATATTATCAGAAGGAAATATACCTGCGTCTTCATCTTTTAAACCAAAACGAGTAGGAAAACCAAAGCGGA

#### Export to pbp.fasta

In [50]:
file = open("pbp.fasta", "w") 
file.write(fasta) 
file.close() 

## 2.8. Ajustes finales

Despues de haberlo pasado por el Clustalo Omega, las secuencias ya están alineadas e igualadas. Al eliminar antes las del alineamiento inverso, tenemos que volver a comprobar e igualar las mutadas de las no mutadas.

In [54]:
f = open("C:\\Users\\migue\\Neoland\\Proyectos\\SpyAR\\data\\blast\\pbp.aln", "r")
f_lista = f.read().split('\n')

In [55]:
f_lista

['>CP035442.1',
 'TAATCTCCTAAAGTAATGGTCATTTTTTTTATTTTTTTCAAGGACTTACCAACATCAACA',
 'CTTTGTTTCATAACACGACCAGAATCTGTTCCTTTAAAGCTGATGTCTATTCCAGTCCAT',
 'TTAGCAAAGGTTTTAACATTGGATTTTGTCCAGCCATACATGTCTGGTACCTCCACAAAA',
 'CGGTCTGATAATATGAGAACTTGTTGGTTTTCTGTTAACGTTTGACCGGGCTGATGCGAT',
 'ACTTTTTTGATCTTGCTGCCAGTACCAAGGACAACTGGCTGGACAAGATTTCGACGCAAC',
 'TCGCTTGATGTCTCACCAGGATTCTTTCCTACAAAGTTTGGTAATTTATAAGTTGTTTGA',
 'CGATTAGCATCTGATACTACTGGCTTAGTTAGTGTATCTTGCATTAAGTATGCTTCTTCC',
 'AATACTGGGTTAACCACATCTTGCCAAAAAAGGGGACCAAAATGTTGTGGTTTAGTCATA',
 'GTAACATACATCAAAAAGTCTGGTTTATCAGCTGGCACCATTGCCACAACCGAATAGACA',
 'TAGTTAGTCAATCCACCATCTTGATAACCACTTCCATCTTCTGAACCAATTTGTGCTGTT',
 'CCTGATTTAACAGCAACAGGTAAATCACCCACTTTAATAATTGGTCCAAATGTTTTTGAA',
 'TAGAGTGTTCCAAACTCAGGGTCTGTTCCTACACCAATCATGTACTGTCTTGTTTCACTA',
 'GCAGCTTTTTTTGATACAGGTTTTCCAACAATTTCTTTATTTGCCGTTCTAAAACTTGCT',
 'GTGTTAGGATCATAAATTTGACTGATAAATTGTGGCTCTAACATCTCACCATTATTAGAA',
 'ATAGCAGTAAAGGCTCTAAGCATTTGAATCTGGGTTACAGAAATCCCTTGACCAAAAGCG',
 'CTCATAG

Como sabemos que cada 39 posiciones obtenemos nuestra secuencia (partiendo de 1 y 39), hacemos lo siguiente para obtener nuestro df definitivo

In [56]:
pbp3 = []

In [57]:
n = 1
m = 39
for i in pbp2:
    seq = ''.join(f_lista[n:m])
    pbp3.append([[w for w in seq],i[2]])
    n += 39
    m += 39

In [58]:
mutados = 0
no_mutados = 0
for i in pbp3:
    if i[1] == 1:
        mutados += 1
    else:
        no_mutados += 1
print('Mutados: ', mutados)
print('No mutados: ', no_mutados)

Mutados:  54
No mutados:  74


Eliminamos no mutados para el balance...

In [59]:
count = 0
while count < (no_mutados-mutados):    
    for i in range(0,len(pbp3)):
        if pbp3[i][1] == 0:
            pbp3.pop(i)
            count += 1
            break

In [60]:
mutados = 0
no_mutados = 0
for i in pbp3:
    if i[1] == 1:
        mutados += 1
    else:
        no_mutados += 1
print('Mutados: ', mutados)
print('No mutados: ', no_mutados)

Mutados:  54
No mutados:  54


# 3. Deep / Machine Learning

## 3.1. Exportando el dataframe con valores originales y valores numéricos

**Valores originales:**

In [67]:
df = pd.DataFrame(pbp3, columns=['X','y'])

In [68]:
df.to_csv('train_cat.csv',header=True,index=False)

In [69]:
df

,X,y
0,"[T, A, A, T, C, T, C, C, T, A, A, A, G, T, A, ...",1
1,"[T, A, A, T, C, T, C, C, T, A, A, A, G, T, A, ...",1
2,"[T, A, A, T, C, T, C, C, T, A, A, A, G, T, A, ...",1
3,"[T, A, A, T, C, T, C, C, T, A, A, A, G, T, A, ...",1
4,"[T, A, A, T, C, T, C, C, T, A, A, A, G, T, A, ...",1
...,...,...
103,"[T, A, A, T, C, T, C, C, T, A, A, A, G, T, A, ...",0
104,"[T, A, A, T, C, T, C, C, T, A, A, A, G, T, A, ...",0
105,"[T, A, A, T, C, T, C, C, T, A, A, A, G, T, A, ...",0
106,"[T, A, A, T, C, T, C, C, T, A, A, A, G, T, A, ...",0


**Valores numéricos:**

Vamos a comprobar los valores únicos. Sabemos que son A, T, G y C, pero con el alineamiento es posible que ClustalOmega haya insertado gaps **('-')** para mejorarlo.

In [70]:
valores = []
for i in range(0,len(pbp3)):
    for j in pbp3[i][0]:
        if j not in valores:
            valores.append(j)
valores

['T', 'A', 'C', 'G']

No hay gaps. Podemos convertir cada uno de los nucleótidos en valores numéricos asignados del siguiente modo:

In [72]:
pbp3_num = []
for i in pbp3:
    seq = []
    for j in i[0]:
        if j == 'A':
            seq.append(0.25)
        elif j == 'C':
            seq.append(0.50)
        elif j == 'G':
            seq.append(0.75)
        else:
            seq.append(1.0)
    pbp3_num.append([seq,i[1]])

In [73]:
df = pd.DataFrame(pbp3_num, columns=['X','y'])

In [74]:
df.to_csv('train_num.csv',header=True,index=False)

In [75]:
df.head()

,X,y
0,"[1.0, 0.25, 0.25, 1.0, 0.5, 1.0, 0.5, 0.5, 1.0...",1
1,"[1.0, 0.25, 0.25, 1.0, 0.5, 1.0, 0.5, 0.5, 1.0...",1
2,"[1.0, 0.25, 0.25, 1.0, 0.5, 1.0, 0.5, 0.5, 1.0...",1
3,"[1.0, 0.25, 0.25, 1.0, 0.5, 1.0, 0.5, 0.5, 1.0...",1
4,"[1.0, 0.25, 0.25, 1.0, 0.5, 1.0, 0.5, 0.5, 1.0...",1


# 3. Construyendo el modelo

Definimos X e y para trabajar con ellos y empezamos a probar diferentes modelos. También vamos a crear una lista donde vamos a ir añadiendo los modelos con sus respectivos scores para compararlos. Es importante recalcar que la métrica que vamos a utilizar es **f1 score**.

In [78]:
X = np.asarray([seq[0] for seq in pbp3_num])
y = np.asarray([seq[1] for seq in pbp3_num])

In [79]:
resultados = []

## Random Forest

In [80]:
from sklearn.ensemble import RandomForestClassifier

Con un RandomForest hay que tener en cuenta muchos parámetros, una combinación de ellos será el más óptimo para nuestro modelo. Por eso, vamos a utilizar GridSearch, una función que te ayuda a obtener esta combinación. Pero primero, hay que elegir los parámetros.

In [81]:
params={'criterion':['gini',"entropy"],
        'max_depth': [5,10,20,30,40,50],# Maxima pofundidad del arbol
        'max_features': [2, 3, 4,5,6], # numero de features a considerar en cada split
        'max_leaf_nodes': [3,5,10,15,20], # maximo de nodos del arbol
        'min_impurity_decrease' : [0.02], # un nuevo nodo se hará si al hacerse decrece la impureza
        'min_samples_split': [2,3,4,5], # mínimo de muestras requeridas para el split en los nodos internos
        'n_estimators':[40,50,60,100,200,300,400]
        }

In [82]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, jaccard_score
from sklearn.metrics import make_scorer

Aquí empieza el GridSearch a buscar los mejores parámetros

In [103]:
%%time
grid_solver = GridSearchCV(estimator = RandomForestClassifier(),
                   param_grid = params, 
                   scoring = make_scorer(f1_score, average ="macro"),
                   cv = 3)
model_result = grid_solver.fit(X,y)

Wall time: 2h 19min 32s


C:\Users\migue\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Y aquí cuáles son y el score (en este caso hemos elegido f1_score).

In [104]:
model_result.best_params_

{'criterion': 'entropy',
 'max_depth': 5,
 'max_features': 2,
 'max_leaf_nodes': 10,
 'min_impurity_decrease': 0.02,
 'min_samples_split': 2,
 'n_estimators': 60}

In [105]:
model_result.best_score_

0.869636830437269

Vamos a entrenar nuestro modelo con esos parámetros...

In [83]:
rf = RandomForestClassifier(criterion='entropy',
                            max_depth= 5,
                            max_features= 2,
                            max_leaf_nodes= 10,
                            min_impurity_decrease= 0.02,
                            min_samples_split= 2,
                            n_estimators= 60)

#### K-Folds Cross Validation

In [84]:
from sklearn.model_selection import cross_val_score

In [85]:
scores = cross_val_score(rf, X, y, cv=5, scoring = make_scorer(f1_score))
print(scores)
print("******"*10)
print("F1-score: ", scores.mean())

[0.83333333 0.70588235 0.95238095 0.95238095 0.82352941]
************************************************************
F1-score:  0.853501400560224


#### Leave One Out

In [86]:
from sklearn.model_selection import LeaveOneOut

In [ ]:
scores = LeaveOneOut(rf, X, y, scoring = make_scorer(f1_score))
print(scores)
print("******"*10)
print("F1-score: ", scores.mean())